In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from python_utils import *
from tqdm.notebook import tqdm, trange
import itertools

Let $r,s,t \ge 4$ be positive integers.

Let $x,y,z$ be positive coprime integers such that $$\delta_r x^r + \delta_s y^s = z^t,$$ where $\delta_r,\delta_s \in \{\pm 1\}$. By Catalan's conjecture, we have $|x|,|y|,|z|\ge 2$.

Assume without loss of generality that $s\ge r\ge t\ge 4$.

### Part 1: compute for assersion (i)

In [ ]:
e0 = 3
def get_ramification_dataset1(l):
    ep_range = dict()
    ep_range['general'] = {'good':[1], 'multi':[1,3,l,3*l]}
    ep_range[2] = {'good':[2], 'multi':[2,6,2*l,6*l]}
    ep_range[3] = {'good':[2,6,8], 'multi':[2,6,2*l,6*l]}
    ep_range[l] = {'good':[l-1,l*(l-1),l*l-1], 'multi':[l-1,3*(l-1),l*(l-1),3*l*(l-1)]}
    return ep_range

# compute log-volume
my_log_volumes1 = dict()
for p in tqdm(primes(200)):
    if p >= 5 and p <= 50:
        my_log_volumes1[p] = compute_log_volume(get_ramification_dataset1, e0, p)
    elif p > 50:
        my_log_volumes1[p] = compute_log_volume_fast(get_ramification_dataset1, e0, p)


  0%|          | 0/46 [00:00<?, ?it/s]

In [124]:
# apos: apostrophe, i.e. the '

def get_impossible_interval_1(log_volumes,S,k,s0,is_l_mid_exponents=False):
    S = sorted(S)
    n = len(S)
    if n < k:
        return []
    S1 = [2]
    e0 = 3
    p_N = 2
    n0 = pow(2,8)

    # define p_*, u_*, n_*
    p0 = S[0]
    u0 = 2 * 4
    n1_S = 2 * p0
    nk_S = 2 * prod(S[:k])
    # N'
    u0_apos = 2 * s0
    if is_l_mid_exponents:
        n1_S_apos = u0_apos * p0
    else:
        n1_S_apos = max(u0_apos, 2*p0)
    # define a_*, b_*
    a1 = 0 
    a2 = 0
    a3 = 0
    a4 = []
    for l in S:
        a1 += (l*l+5*l)/(l*l+l-12)*(1-1/e0/l)
        a2 += log_volumes[l] 
        a3 += log(l)
        a4.append((l+5)*(l-1)/(l*l+l-12)/e0)
    a1 = a1 / n * 6
    a2 = a2 / n * 6
    a3 = a3
    a4 = min(a4) * 6
    a5 = sum([log(p) for p in S1])
    #  define b_1, b'_1, b_2, impossible inteval
    b1 = max(k/n + (a1-a4) / n1_S, a1/u0)
    b1_apos = max(k/n + (a1-a4) / n1_S_apos, a1/u0_apos)
    b2 = a1/u0 * log(n0) + a2 + a1*a3 + (a1-a4)*a5
    if b1 > 1 or b1_apos >= 1:
        return []
    upper_bound = nk_S * log(p_N)
    lower_bound = b2 / (1-b1_apos)
    if lower_bound > upper_bound:
        return []
    return [lower_bound,upper_bound]


def get_upper_bound_1(log_volumes,s0):
    my_primes = [p for p in primes(200) if p >= 11]
    # choice of possible set S
    intervals = []
    arr_S = []
    for n in range(2,20):
        for k in range(len(my_primes)-n):
            arr_S.append(my_primes[k:k+n])
    # impossible inteval for log(N')
    for S in arr_S:
        for k in [2,3,4,5]:
            interval = get_impossible_interval_1(log_volumes,S,k,s0,is_l_mid_exponents=False)
            if len(interval)==2:
                intervals.append(interval)
    res = merge_intervals(intervals)
    if len(res) == 0:
        return -999
    return (res[-1][0] + log(pow(2,8))) / 2

# upper bounds for different s0
for s0 in [i for i in range(4,15)] + [100*i for i in range(1,7)]:
    bound = get_upper_bound_1(my_log_volumes1,s0)
    print(s0,f"{bound + 1e-3: .3f}")



4  1692.823
5  863.550
6  667.141
7  570.957
8  545.217
9  541.231
10  538.040
11  538.040
12  538.040
13  538.040
14  538.040
100  442.557
200  433.291
300  430.289
400  428.803
500  427.917
600  427.328


Since $2^s \le \log(y^s) < 427.5 / \log(2) < 617$, we have $t = \min \{ r,s,t \} \le 616$.


In [8]:
427.5 / log(2)  

616.7521299800319

### Part 2: compute for assersion (ii)

In [ ]:
e0 = 1
def get_ramification_dataset3(l):
    ep_range = dict()
    ep_range['general'] = {'good':[1], 'multi':[1,l]}
    ep_range[2] = {'good':[2], 'multi':[2,2*l]}
    ep_range[l] = {'good':[l-1,l*(l-1),l*l-1], 'multi':[l-1,l*(l-1)]}
    return ep_range

# compute log-volume
my_log_volumes3 = dict()
for p in tqdm(primes(200)):
    if p >= 5 and p <= 50:
        my_log_volumes3[p] = compute_log_volume(get_ramification_dataset3, e0, p)
    elif p > 50:
        my_log_volumes3[p] = compute_log_volume_fast(get_ramification_dataset3, e0, p)


  0%|          | 0/46 [00:00<?, ?it/s]

In [126]:
def get_impossible_interval_2(log_volumes,S,k,s0,is_l_mid_exponents=False):
    S = sorted(S)
    n = len(S)
    if n < k:
        return []
    S1 = []
    e0 = 1
    p_N = 3
    n0 = 1

    # define p_*, u_*, n_*
    p0 = S[0]
    u0 = 2 * 4
    n1_S = 2 * p0
    nk_S = 2 * prod(S[:k])
    # N'
    u0_apos = 2 * s0
    if is_l_mid_exponents:
        n1_S_apos = u0_apos * p0
    else:
        n1_S_apos = max(u0_apos, 2*p0)
    # define a_*, b_*
    a1 = 0 
    a2 = 0
    a3 = 0
    a4 = []
    for l in S:
        a1 += (l*l+5*l)/(l*l+l-12)*(1-1/e0/l)
        a2 += log_volumes[l] + log(2) # by the definition of Vol(l)
        a3 += log(l)
        a4.append((l+5)*(l-1)/(l*l+l-12)/e0)
    a1 = a1 / n * 6
    a2 = a2 / n * 6
    a3 = a3
    a4 = min(a4) * 6
    a5 = sum([log(p) for p in S1])
    #  define b_1, b'_1, b_2, impossible inteval
    b1 = max(k/n + (a1-a4) / n1_S, a1/u0)
    b1_apos = max(k/n + (a1-a4) / n1_S_apos, a1/u0_apos)
    b2 = a1/u0 * log(n0) + a2 + a1*a3 + (a1-a4)*a5
    if b1 > 1 or b1_apos >= 1:
        return []
    upper_bound = nk_S * log(p_N)
    lower_bound = b2 / (1-b1_apos)
    if lower_bound > upper_bound:
        return []
    return [lower_bound,upper_bound]


def get_upper_bound_2(log_volumes,s0):
    my_primes = [p for p in primes(200) if p >= 11]
    # choice of possible set S
    intervals = []
    arr_S = []
    for n in range(2,20):
        for k in range(len(my_primes)-n):
            arr_S.append(my_primes[k:k+n])
    # impossible inteval for log(N')
    for S in arr_S:
        for k in [2,3,4,5]:
            interval = get_impossible_interval_2(log_volumes,S,k,s0,is_l_mid_exponents=False)
            if len(interval)==2:
                intervals.append(interval)
    res = merge_intervals(intervals)
    if len(res) == 0:
        return -999
    return res[-1][0] / 2

# upper bounds for different s0
for s0 in [i for i in range(4,15)]:
    bound = get_upper_bound_2(my_log_volumes3,s0)
    print(s0,f"{bound + 1e-3: .3f}")



4  804.308
5  393.280
6  310.066
7  266.195
8  255.221
9  245.858
10  245.408
11  245.408
12  245.408
13  245.408
14  245.408


When $s_0 \ge 8$, we have $h < 257$. We shall compute more when at least one of $r,s,t$ equals $7$.

In [77]:
def get_impossible_interval_3(log_volumes,S,k,s0,r,s,t):
    # s0 equals one of r,s,t, l \nmid rst
    S = sorted(S)
    n = len(S)
    if n < k:
        return []
    S1 = []
    e0 = 1
    p_N = 3
    n0 = 3

    # define p_*, u_*, n_*
    p0 = S[0]
    u0 = 2 * min(r,s,t)
    n1_S = u0 * p0
    nk_S = u0 * prod(S[:k])
    # N'
    u0_apos = 2 * s0
    n1_S_apos = u0_apos * p0
    # define a_*, b_*
    a1 = 0 
    a2 = 0
    a3 = 0
    a4 = []
    for l in S:
        a1 += (l*l+5*l)/(l*l+l-12)*(1-1/e0/l)
        a2 += log_volumes[l] + log(2) # by the definition of Vol(l)
        a3 += log(l)
        a4.append((l+5)*(l-1)/(l*l+l-12)/e0)
    a1 = a1 / n * 6
    a2 = a2 / n * 6
    a3 = a3
    a4 = min(a4) * 6
    a5 = sum([log(p) for p in S1])
    #  define b_1, b'_1, b_2, impossible inteval
    b1 = max(k/n + (a1-a4) / n1_S, a1/u0)
    b1_apos = max(k/n + (a1-a4) / n1_S_apos, a1/u0_apos)
    b2 = a1/u0 * log(n0) + a2 + a1*a3 + (a1-a4)*a5
    if b1 > 1 or b1_apos >= 1:
        return []
    upper_bound = nk_S * log(p_N)
    lower_bound = b2 / (1-b1_apos)
    if lower_bound > upper_bound:
        return []
    return [lower_bound,upper_bound]


def get_upper_bound_3(log_volumes,s0,r,s,t):
    my_primes = [p for p in primes(200) if p >= 11 and r*s*t % p != 0]
    # choice of possible set S
    intervals = []
    arr_S = []
    for n in range(2,20):
        for k in range(len(my_primes)-n):
            arr_S.append(my_primes[k:k+n])
    # impossible inteval for log(N')
    for S in arr_S:
        for k in [2,3,4,5]:
            interval = get_impossible_interval_3(log_volumes,S,k,s0,r,s,t)
            if len(interval)==2:
                intervals.append(interval)
    res = merge_intervals(intervals)
    if len(res) == 0:
        return -999
    return res[-1][0] / 2




When $r,s,t$ are not divided by $11,13,17,19$, we have $2h < 510$, hence $h < 255$.

In [111]:
arr_S = list(itertools.combinations([11,13,17,19,23],4))
for S in arr_S:
    print(S, get_impossible_interval_3(my_log_volumes3,S,k=2,s0=7,r=4,s=4,t=7))

(11, 13, 17, 19) [499.4682095570748, 1256.8124582363175]
(11, 13, 17, 23) [505.9595024651539, 1256.8124582363175]
(11, 13, 19, 23) [508.5555975852461, 1256.8124582363175]
(11, 17, 19, 23) [507.6678359602779, 1643.5239838474922]
(13, 17, 19, 23) [504.563527302414, 1942.346526365218]


In [113]:
max_r = 616

s0 = 7
for r in [11,13,17,19,23]:
    for s in [11,13,17,19,23]:
        bound = get_upper_bound_3(my_log_volumes3,s0,r,s,7)
        print(r,s,bound)


11 11 251.8488077467494
11 13 263.04134827897263
11 17 260.62746199519466
11 19 259.4945211061184
11 23 256.56601115329437
13 11 263.04134827897263
13 13 253.3891495162904
13 17 262.33233349965644
13 19 261.17579500806045
13 23 258.19181494338744
17 11 260.62746199519466
17 13 262.33233349965644
17 17 253.81880358700298
17 19 261.8021462485959
17 23 258.7375732455667
19 11 259.4945211061184
19 13 261.17579500806045
19 17 261.8021462485959
19 19 252.51612171462799
19 23 257.484126557055
23 11 256.56601115329437
23 13 258.19181494338744
23 17 258.7375732455667
23 19 257.484126557055
23 23 249.26381061945455


In [119]:
for s0 in range(9,20):
    print(s0,f"{exp(268*2/3/s0):.2e}")

9 4.18e+08
10 5.75e+07
11 1.13e+07
12 2.93e+06
13 9.31e+05
14 3.49e+05
15 1.49e+05
16 7.07e+04
17 3.67e+04
18 2.05e+04
19 1.21e+04


In [9]:
my_primes = [11,13,17,19,23,29,31,37]
a1_dict = dict()
a2_dict = dict()
for p in my_primes:
    a1_dict[p] = 3*(p*p+5*p)/(p*p+p-12)*(1-1/3/p)
    a2_dict[p] = max(3*my_log_volumes1[p] + a1_dict[p]*log(2), 3*my_log_volumes2[p] + a1_dict[p]*log(2*p))
    print(p, a1_dict[p], a2_dict[p])
print(17, a2_dict[17]+4*log(3))

11 4.2666666666666675 131.28523392383883
13 4.023529411764706 137.34703569022344
17 3.7414965986394555 155.48847940816694
19 3.652173913043478 163.917676856701
23 3.525925925925926 181.54198264105023
29 3.4079254079254078 209.57386002385223
31 3.379591836734694 218.89589134822631
37 3.3142037302725966 247.3515648511106
17 159.88292856283937


In [10]:
864 / log(2) / 5, 1693 / log(2) / 4, (164/log(2)+4),(182/log(2)+4),(219/log(2)+4)


(249.2977030656129,
 610.6206760562537,
 240.60198670579,
 266.57049744179136,
 319.950213954683)

In [11]:
164/log(2)+4,182/log(2)+4, 1693/(17*17*log(2))

(240.60198670579, 266.57049744179136, 8.45149724645334)

Since $r\ge 5$, note that by Proposition \ref{2-1-prop-bound}, we have $2^{r_2 r} \le x^ry^sz^t \le e^{668}$, hence $r_2 \le \frac{864}{r\cdot \log(2)} < 17^2$.